--- Day 13: Transparent Origami ---

You reach another volcanically active part of the cave. It would be nice if you could do some kind of thermal imaging so you could tell ahead of time which caves are too hot to safely enter.

Fortunately, the submarine seems to be equipped with a thermal camera! When you activate it, you are greeted with:

Congratulations on your purchase! To activate this infrared thermal imaging
camera system, please enter the code found on page 1 of the manual.

Apparently, the Elves have never used this feature. To your surprise, you manage to find the manual; as you go to open it, page 1 falls out. It's a large sheet of transparent paper! The transparent paper is marked with random dots and includes instructions on how to fold it up (your puzzle input). For example:

```
6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0
```

```
fold along y=7
fold along x=5
```

The first section is a list of dots on the transparent paper. 0,0 represents the top-left coordinate. The first value, x, increases to the right. The second value, y, increases downward. So, the coordinate 3,0 is to the right of 0,0, and the coordinate 0,7 is below 0,0. The coordinates in this example form the following pattern, where # is a dot on the paper and . is an empty, unmarked position:

```
...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........
```

Then, there is a list of fold instructions. Each instruction indicates a line on the transparent paper and wants you to fold the paper up (for horizontal y=... lines) or left (for vertical x=... lines). In this example, the first fold instruction is fold along y=7, which designates the line formed by all of the positions where y is 7 (marked here with -):

```
...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
-----------
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........
```

Because this is a horizontal line, fold the bottom half up. Some of the dots might end up overlapping after the fold is complete, but dots will never appear exactly on a fold line. The result of doing this fold looks like this:

```
#.##..#..#.
#...#......
......#...#
#...#......
.#.#..#.###
...........
...........
```

Now, only 17 dots are visible.

Notice, for example, the two dots in the bottom left corner before the transparent paper is folded; after the fold is complete, those dots appear in the top left corner (at 0,0 and 0,1). Because the paper is transparent, the dot just below them in the result (at 0,3) remains visible, as it can be seen through the transparent paper.

Also notice that some dots can end up overlapping; in this case, the dots merge together and become a single dot.

The second fold instruction is fold along x=5, which indicates this line:

```
#.##.|#..#.
#...#|.....
.....|#...#
#...#|.....
.#.#.|#.###
.....|.....
.....|.....
```

Because this is a vertical line, fold left:

```
#####
#...#
#...#
#...#
#####
.....
.....
```

The instructions made a square!

The transparent paper is pretty big, so for now, focus on just completing the first fold. After the first fold in the example above, 17 dots are visible - dots that end up overlapping after the fold is completed count as a single dot.

How many dots are visible after completing just the first fold instruction on your transparent paper?


In [ ]:
#load "../Shared/regexp.fsx"
open RegexpMatch

let testInput = """
6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5
"""

type Instructions = bool[,] * seq<bool * int>

let parse (input:string) =
    let parts = input.Replace("\r", "").Split("\n\n")

    let coords =
        parts.[0].Split("\n")
        |> Seq.filter ( (<>) "" )
        |> Seq.map (
            function
            | Regex @"(\d+)\s?,\s?(\d+)" [x; y] -> (int x, int y)
            | token -> failwithf "Unspected input '%s'" token
        )

    let width = coords |> Seq.map fst |> Seq.max
    let height = coords |>Seq.map snd |> Seq.max
    let map = Array2D.zeroCreate<bool> (height+1) (width+1)

    coords
    |> Seq.iter (fun (x,y) -> 
        map[y,x] <- true
    )

    let folds =
        parts.[1].Split("\n")
        |> Seq.filter( (<>) "" )
        |> Seq.map(
            function
            | Regex @"fold along (x|y)\s?=\s?(\d+)" [axis; amount] -> (axis = "x", int amount)
            | token -> failwithf "Unepexcted input '%s'" token
        )

    (map, folds)

testInput |> parse

Item1,Item2
"[ False, False, False, True, False, False, True, False, False, True, False, False, False, False, False, True, False, False, False, False ... (more) ]","[ ( False, 7 ), ( True, 5 ) ]"


In [ ]:
let printMap (map: bool[,]) =
    for row = 0 to (map.GetLength(0) - 1 ) do
        let mutable s = ""
        for col = 0 to (map.GetLength(1) - 1 ) do
            s <- s + (if map[row,col] then "#" else "." )
        printfn "%s" s

testInput |> parse |> fst |> printMap

...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........


In [ ]:
let foldX (where:int) (map:bool[,])  =
    let width = where
    let height = map.GetLength(0)
    let newMap = Array2D.zeroCreate<bool> height width 
    let maxW = Math.Min(width*2, map.GetLength(1)-1)
    for row=0 to (height-1) do
        for col=0 to (width-1) do
            newMap[row, col] <- map[row, col]
        for col=width+1 to maxW do
            newMap[row, width*2-col] <- map[row, col] || newMap[row, width*2-col]

    newMap            

testInput |> parse |> fst |> foldX 5 |> printMap


.#.##
....#
.....
#....
#.##.
.....
.....
.....
.....
.....
.##.#
....#
#...#
#....
#.#..


In [ ]:
let foldY (where:int) (map: bool[,]) =
    let width = map.GetLength(1)
    let height = where
    let newMap = Array2D.zeroCreate<bool> height width
    let maxH = Math.Min(height*2, map.GetLength(0) - 1 )
    for row=0 to (height-1) do
        for col=0 to (width-1) do
            newMap[row, col] <- map[row, col]
    for row=(height+1) to maxH do
        let targetRow = height*2 - row
        for col=0 to (width-1) do
            newMap[targetRow, col] <- map[row, col] || newMap[targetRow, col]

    newMap

testInput |> parse |> fst |> foldY 7 |> printMap

#.##..#..#.
#...#......
......#...#
#...#......
.#.#..#.###
...........
...........


In [ ]:
testInput |> parse |> fst |> foldY 7 |> foldX 5 |> printMap

#####
#...#
#...#
#...#
#####
.....
.....


In [ ]:
let countDots (map: bool[,]) =
    let mutable result = 0

    map
    |> Array2D.iter ( fun value -> 
        if value then
            result <- result + 1
    )
    result

testInput |> parse |> fst |> foldY 7 |> foldX 5 |>  countDots

16

In [ ]:
let task = System.IO.File.ReadAllText "task.txt"

task |> parse |> ( fun (map, folds) -> 
    let folds2 = folds |> Array.ofSeq 
    if fst folds2[0] then
        map |> foldX (snd folds2[0])
    else
        map |> foldY (snd folds2[0])
) |> countDots

807

--- Part Two ---

Finish folding the transparent paper according to the instructions. The manual says the code is always eight capital letters.

What code do you use to activate the infrared thermal imaging camera system?


In [ ]:
let foldAll (map: bool[,]) (folds:seq<bool * int>) =
    folds
    |> Seq.fold (fun map (isX, where) ->
            if isX then
                foldX where map
            else
                foldY where map
        )
        map

testInput |> parse ||> foldAll |> printMap

#####
#...#
#...#
#...#
#####
.....
.....


In [ ]:
task |> parse ||> foldAll |> printMap

#.....##..#..#.####..##..#..#.####...##.
#....#..#.#..#.#....#..#.#..#.#.......#.
#....#....####.###..#....#..#.###.....#.
#....#.##.#..#.#....#.##.#..#.#.......#.
#....#..#.#..#.#....#..#.#..#.#....#..#.
####..###.#..#.####..###..##..####..##..
